In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import requests
from pathlib import Path

In [3]:
CLEANPATH = Path.cwd().parent / 'clean-data' 

In [4]:
df_province = pd.read_csv(CLEANPATH /'province_cumu_clean.csv',index_col=0)

In [72]:
df_province.query("date=='2020-12-08'")

,date,province,total,admitted,deaths
3240,2020-12-08,Drenthe,8458,216,116
3241,2020-12-08,Flevoland,12685,404,147
3242,2020-12-08,Friesland,9665,213,142
3243,2020-12-08,Gelderland,58178,2079,1103
3244,2020-12-08,Groningen,9625,153,55
3245,2020-12-08,Limburg,28640,1810,913
3246,2020-12-08,Noord-Brabant,91530,3561,2106
3247,2020-12-08,Noord-Holland,102033,2824,1352
3248,2020-12-08,Overijssel,35342,764,564
3249,2020-12-08,Utrecht,48333,1486,733


In [5]:
province_list = df_province['province'].unique()

array(['Drenthe', 'Flevoland', 'Friesland', 'Gelderland', 'Groningen',
       'Limburg', 'Noord-Brabant', 'Noord-Holland', 'Overijssel',
       'Utrecht', 'Zeeland', 'Zuid-Holland'], dtype=object)

In [9]:
#fig = px.line(df_province.query("province=='Noord-Holland'"), x='date', y=['total', 'admitted', 'deaths'])
#fig.show()

In [42]:
df_province['date']

0       2020-03-13
1       2020-03-13
2       2020-03-13
3       2020-03-13
4       2020-03-13
           ...    
3247    2020-12-08
3248    2020-12-08
3249    2020-12-08
3250    2020-12-08
3251    2020-12-08
Name: date, Length: 3252, dtype: object

In [28]:
##Line plot of cumulative reported covid cases per province in the Netherlands

fig=go.Figure()

for prov in province_list:
    fig.add_trace(
        go.Scatter(
            x=df_province.query("province=='{}'".format(prov))['date'],
            y=df_province.query("province=='{}'".format(prov))['total'],
            name = prov
        )
    )

button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': ((df_province['province'].unique())==(province_list)),
                               'title': 'All',
                               'showlegend':True}])    
    
def create_layout_button(prov):
        return dict(label = prov,
                    method = 'update',
                    args = [{'visible': [p == prov for p in province_list],
                             'title': prov,
                             'showlegend': True}])
    
prov_series=pd.Series(province_list)

fig.update_layout(
    width=900,
    height=400,
    autosize=True,
    margin=dict(t=20, b=0, l=0, r=0),
    template="plotly_white",)
    
fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = [button_all] + list(prov_series.map(lambda prov: create_layout_button(prov)))
            )
        ])
    
fig.show()

In [33]:
###maps

In [34]:
import geopandas

In [64]:
nl_geojson=r"the-netherlands.geojson"

In [54]:
nl_geojson

PosixPath('/Users/srdpsrknth/OneDrive/CovidDash/covid19dash/external/nl_provinces.geojson')

In [37]:
provs = json.load(nl_geojson)
provs["features"][0]

AttributeError: 'PosixPath' object has no attribute 'read'

In [78]:
 fig = px.choropleth(df_province.query("date=='2020-12-08'"), geojson=nl_geojson, locations='province',
                     featureidkey="properties.name",
                     color=df_province.query("date=='2020-12-08'")['total'],
                     color_continuous_scale="Brwnyl",
                     scope="europe",
                    )
fig.update_geos(fitbounds="locations", visible=False)
fig.show()